## This demo showcases the implementation of user story 530

All initiated jobs for the staging process should be stored in a persistent PostgreSQL database. This ensures that even if the rs-server-staging pod is restarted, users can still check the status of all previously submitted jobs.

In [1]:
import requests
import os
import pprint
import time
# Init environment before running a demo notebook.
from resources.utils import *
import logging

logger = logging.getLogger(__name__)  # Utilise le nom du module actuel
logger.setLevel(logging.INFO)  # Définit le niveau du logger à INFO
print(f" --------- RSPY_APPLY_STAGING_ENDPOINTS_VALIDATION vaut {os.getenv('RSPY_APPLY_STAGING_ENDPOINTS_VALIDATION')}")
pp = pprint.PrettyPrinter(indent=2, width=80, sort_dicts=False, compact=True)
session = requests.Session()
auxip_client, cadip_client, stac_client, staging_client = init_demo()

if os.getenv("RSPY_LOCAL_MODE") == "1":
    href_cadip = "http://rs-server-cadip:8000"
    href_adgs = "http://rs-server-adgs:8000"
else:
    href_cadip = href_adgs = os.environ["RSPY_WEBSITE"]
    session.cookies.set ("session", os.environ["RSPY_OAUTH2_COOKIE"])

cadip_collection_id = "cadip_sentinel1" #"cadip_s1A"
adgs_collection_id = "adgs" #"aux_interval"
TIMEOUT = 10

Auxip service: http://rs-server-adgs:8000
CADIP service: http://rs-server-cadip:8000
Catalog service: http://rs-server-catalog:8000
Staging service: http://rs-server-staging:8000


In [2]:
# Create a test collection 
collection = create_test_collection()

### Check the newly created collection with the rs-server-catalog. It should be empty.

In [3]:
# Check the catalog for my_test_collection
collection = stac_client.get_collection(TEST_COLLECTION)
for item in collection.get_items():
    print(f"Item {item.id} has {len(item.assets)} assets")

### Getting all the sessions from the input collection found in the configuration of CADIP station

In [4]:
# TODO: CadipClient should inherit pystac_client.Client so we can use its wrapped functions
result = session.get(f"{href_cadip}/cadip/collections/{cadip_collection_id}/items")
items_collection_cadip = result.json()

# TODO: For now we stage less than ten items because of the bug of the wrong pagination url - we stage one item from the cadip station
items_collection_cadip["features"] = [items_collection_cadip["features"][0]]

assert items_collection_cadip.get("type") == "FeatureCollection"
assert len(items_collection_cadip.get("features")) > 0
for item in items_collection_cadip.get("features"):
    print(f"Session {item.get('id')} has {len(item.get('assets'))} assets with datetime {item.get('properties').get('datetime')}")

Session S1A_20231120061537234567 has 60 assets with datetime 2023-11-20T06:15:37.234Z


### Getting all the assets from the input collection found in the configuration of ADGS station

We will deliberately alter the value of a download link for one of the files, causing the staging process to fail at a certain point.

In [5]:
# TODO: AuxipClient should inherit pystac_client.Client so we can use its wrapped functions
result = session.get(f"{href_adgs}/auxip/collections/{adgs_collection_id}/items")
items_collection_adgs = result.json()

# TODO: For now we stage less than ten items because of the bug of the wrong pagination url - we stage nine items from the auxip station
items_collection_adgs["features"] = items_collection_adgs["features"][:9] 

assert items_collection_adgs.get("type") == "FeatureCollection"
assert len(items_collection_adgs.get("features")) > 0

#pprint.PrettyPrinter(indent=4).pprint(items_collection_adgs)
for item in items_collection_adgs.get("features"):
    print(f"AUXIP asset {item.get('id')} has datetime {item.get('properties').get('datetime')}")

AUXIP asset S1A_OPER_AUX_RESORB_OPOD_20231218T110702_V20231218T071044_20231218T102814.EOF has datetime 2023-12-18T23:27:56.465Z
AUXIP asset S1A_OPER_AUX_RESORB_OPOD_20230405T110702_V20230405T071044_20230405T102814.EOF has datetime 2023-04-05T01:53:47.843Z
AUXIP asset S2__OPER_AUX_ECMWFD_PDMC_20230216T120000_V20190217T090000_20190217T210000.TGZ has datetime 2023-02-16T12:55:35.361Z
AUXIP asset S1A_OPER_AUX_PREORB_OPOD_20230122T062732_V20230122T062732_20230122T062732.EOF has datetime 2023-01-22T21:06:06.426Z
AUXIP asset S1A_OPER_MPL_ORBPRE_20210214T021411_20210221T021411_0001.EOF has datetime 2021-02-14T20:16:05.577Z
AUXIP asset S1A_OPER_AUX_OBMEMC_PDMC_20210123T000000.xml has datetime 2021-01-23T00:52:18.805Z
AUXIP asset S1A_OPER_AUX_PREORB_OPOD_20201002T062732_V20201002T053140_20201002T120640.EOF has datetime 2020-10-02T16:17:25.992Z
AUXIP asset S1A_OPER_MPL_ORBSCT_20200829T150704_99999999T999999_0025.EOF has datetime 2020-08-29T14:33:30.117Z
AUXIP asset S2__OPER_AUX_ECMWFD_PDMC_202002

Check all existing processes and then information about the staging process

In [6]:
# Returns list of all available processes from config.
processes = staging_client.get_processes()
print(processes)

TEST
DO_VALIDATE VAUT: 0
{'processes': [{'name': 'staging', 'processor': 'Staging'}]}


In [7]:
# Should return info about the staging process.
process_info = staging_client.get_process("staging")
print(process_info)

{'type': 'process', 'processor': {'name': 'Staging'}}


### Check the jobs table

In [8]:
jobs = staging_client.get_jobs()
if jobs.get("numberMatched") > 0:
    delete_jobs = True
    if cluster_mode == "1":
        delete_jobs = input(f"There are {jobs.get('numberMatched')} jobs in the table. Do you want to delete them all (y/n)?").lower().strip() == 'y'
    if delete_jobs:
        print("Deleting all the jobs...")
        for job in jobs.get("jobs"):
            delete_response = staging_client.delete_job(job.get('identifier'))
        # Check that the jobs have been deleted
        get_response = staging_client.get_jobs()
        print(f"Existing jobs: {get_response}")

There are 36 jobs in the table. Do you want to delete them all (y/n)? n


### Starting 2 staging processes, one from the CADIP station and one from the ADGS station
The staging process from the ADGS station is expected to fail because one of the assets contains an incorrect download link for the file.

In [9]:
staging_resp_list = []
for items in [items_collection_cadip, items_collection_adgs]:
    staging_resp_list.append(staging_client.run_staging(items, TEST_COLLECTION))

In [10]:
items

{'type': 'FeatureCollection',
 'features': [{'bbox': [-180.0, -90.0, 180.0, 90.0],
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-180, -90],
      [180, -90],
      [180, 90],
      [-180, 90],
      [-180, -90]]]},
   'properties': {'datetime': '2023-12-18T23:27:56.465Z',
    'created': '2023-12-18T23:27:56.465Z',
    'start_datetime': '2023-12-18T23:27:56.465Z',
    'end_datetime': '2023-12-18T23:28:06.465Z',
    'platform': 'sentinel-1a',
    'constellation': 'sentinel-1',
    'auxip:id': '035c9267-9308-4c40-bad4-167b3a0622b5',
    'published': '2023-12-18T23:27:56.465Z',
    'product:type': 'OPER_AUX_RESORB_OPOD',
    'processing:datetime': '2023-12-18T00:00:00.000Z',
    'processing:facility': 'FOS'},
   'id': 'S1A_OPER_AUX_RESORB_OPOD_20231218T110702_V20231218T071044_20231218T102814.EOF',
   'stac_version': '1.0.0',
   'assets': {'S1A_OPER_AUX_RESORB_OPOD_20231218T110702_V20231218T071044_20231218T102814.EOF': {'href': 'http://adgs-station:5000/Pro

In [11]:
timeout = 120
for resp in staging_resp_list:
    while timeout > 0:
        job_info = staging_client.get_job_info(resp["status"]["started"])
        ### TODO: a remplacer par la ligne suivant après la mise à jours de rs-server-staging
        ###job_info = staging_client.get_job_info(resp["jobID"])
        pprint.PrettyPrinter(indent=4).pprint(job_info)
        print("\n")
        ### TODO: a remplacer par la ligne suivant après la mise à jours de rs-server-staging
        ###if resp["status"] == "successful":
        if resp["status"] == "FINISHED":
            print("Job COMPLETED")
            break
        ### TODO: a remplacer par la ligne suivant après la mise à jours de rs-server-staging
        ###if resp["status"] == "failed":
        if resp["status"] == "FAILED":
            print("Job FAILED")
            break
        time.sleep(2)
        timeout -= 2

{   'created_at': '2025-01-20T10:07:35.041999',
    'detail': 'In progress',
    'identifier': '5efca69f-ecef-4201-908f-7c71b170d378',
    'progress': 10.0,
    'status': 'IN_PROGRESS',
    'updated_at': '2025-01-20T10:07:35.668235'}


{   'created_at': '2025-01-20T10:07:35.041999',
    'detail': 'In progress',
    'identifier': '5efca69f-ecef-4201-908f-7c71b170d378',
    'progress': 100.0,
    'status': 'IN_PROGRESS',
    'updated_at': '2025-01-20T10:07:36.941253'}


{   'created_at': '2025-01-20T10:07:35.041999',
    'detail': 'In progress',
    'identifier': '5efca69f-ecef-4201-908f-7c71b170d378',
    'progress': 100.0,
    'status': 'IN_PROGRESS',
    'updated_at': '2025-01-20T10:07:36.941253'}


{   'created_at': '2025-01-20T10:07:35.041999',
    'detail': 'Finished',
    'identifier': '5efca69f-ecef-4201-908f-7c71b170d378',
    'progress': 100.0,
    'status': 'FINISHED',
    'updated_at': '2025-01-20T10:07:40.547435'}


{   'created_at': '2025-01-20T10:07:35.041999',
    'detail

### Check the catalog for my_test_collection. Four items should be present now

In [12]:
user = os.environ["JUPYTERHUB_USER"] if cluster_mode else os.environ["RSPY_HOST_USER"]
result = session.get(f"{stac_client.href_catalog}/catalog/collections/{user}:{TEST_COLLECTION}/items")
catalog_collection = result.json()
for item in catalog_collection.get("features"):
    print(f"Item {item.get('id')} has {len(item.get('assets'))} assets")

Item S1A_OPER_AUX_RESORB_OPOD_20231218T110702_V20231218T071044_20231218T102814.EOF has 1 assets
Item S1A_20231120061537234567 has 60 assets
Item S1A_OPER_AUX_RESORB_OPOD_20230405T110702_V20230405T071044_20230405T102814.EOF has 1 assets
Item S2__OPER_AUX_ECMWFD_PDMC_20230216T120000_V20190217T090000_20190217T210000.TGZ has 1 assets
Item S1A_OPER_AUX_PREORB_OPOD_20230122T062732_V20230122T062732_20230122T062732.EOF has 1 assets
Item S1A_OPER_MPL_ORBPRE_20210214T021411_20210221T021411_0001.EOF has 1 assets
Item S1A_OPER_AUX_OBMEMC_PDMC_20210123T000000.xml has 1 assets
Item S1A_OPER_AUX_PREORB_OPOD_20201002T062732_V20201002T053140_20201002T120640.EOF has 1 assets
Item S1A_OPER_MPL_ORBSCT_20200829T150704_99999999T999999_0025.EOF has 1 assets
Item S2__OPER_AUX_ECMWFD_PDMC_20200216T120000_V20190217T090000_20190217T210000.TGZ has 1 assets


In [13]:
# Check the catalog for my_test_collection
result = list(stac_client.get_collection(TEST_COLLECTION).get_items())
print (f"{len(result)} items before removing")
for item in result:
    print(f"Item {item.id} has {len(item.assets)} assets")

10 items before removing
Item S1A_OPER_AUX_RESORB_OPOD_20231218T110702_V20231218T071044_20231218T102814.EOF has 1 assets
Item S1A_20231120061537234567 has 60 assets
Item S1A_OPER_AUX_RESORB_OPOD_20230405T110702_V20230405T071044_20230405T102814.EOF has 1 assets
Item S2__OPER_AUX_ECMWFD_PDMC_20230216T120000_V20190217T090000_20190217T210000.TGZ has 1 assets
Item S1A_OPER_AUX_PREORB_OPOD_20230122T062732_V20230122T062732_20230122T062732.EOF has 1 assets
Item S1A_OPER_MPL_ORBPRE_20210214T021411_20210221T021411_0001.EOF has 1 assets
Item S1A_OPER_AUX_OBMEMC_PDMC_20210123T000000.xml has 1 assets
Item S1A_OPER_AUX_PREORB_OPOD_20201002T062732_V20201002T053140_20201002T120640.EOF has 1 assets
Item S1A_OPER_MPL_ORBSCT_20200829T150704_99999999T999999_0025.EOF has 1 assets
Item S2__OPER_AUX_ECMWFD_PDMC_20200216T120000_V20190217T090000_20190217T210000.TGZ has 1 assets


### Check the jobs table

In [14]:
post_response = staging_client.get_jobs()
pprint.PrettyPrinter(indent=4).pprint(jobs)
print("\n\nSummary")

for job in jobs.get("jobs"):    
    job_results = staging_client.get_job_results(job["identifier"])
    print(f"{job['identifier']} status is {job_results}")

{   'jobs': [   {   'created_at': '2025-01-19T15:08:18.650199',
                    'detail': 'Finished',
                    'identifier': '66ad90a6-9044-4851-8ec3-f79477f801d8',
                    'progress': 100.0,
                    'status': 'FINISHED',
                    'updated_at': '2025-01-19T15:08:21.050272'},
                {   'created_at': '2025-01-19T15:08:18.760538',
                    'detail': 'Finished',
                    'identifier': '6a8935bc-8345-4200-837b-b6de422f7ead',
                    'progress': 100.0,
                    'status': 'FINISHED',
                    'updated_at': '2025-01-19T15:08:21.835632'},
                {   'created_at': '2025-01-19T15:07:47.952491',
                    'detail': 'Finished',
                    'identifier': '56da04dc-95cc-45ae-a30b-273cd8df8be3',
                    'progress': 100.0,
                    'status': 'FINISHED',
                    'updated_at': '2025-01-19T15:07:49.888862'},
                {   'c

### Delete the whole collection

In [15]:
result = stac_client.remove_collection(TEST_COLLECTION)
assert result.json()["deleted collection"] == TEST_COLLECTION
pp.pprint(result.json())

{'deleted collection': 'my_test_collection'}
